This notebook is based on [this example](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/semantic-search.ipynb#scrollTo=Fqo_hMRZiubM)

In [ ]:
!pip install faiss-cpu langchain langchain-community langgraph langchain-openai langchain-anthropic -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 20.2 MB/s eta 0:00:00


In [ ]:
from langchain.tools.retriever import create_retriever_tool

from langgraph.prebuilt import create_react_agent
from google.colab import userdata


In [ ]:
!git clone https://github.com/RadchaneepornC/OfficeBuddyPrime_LLMAgentic

Cloning into 'OfficeBuddyPrime_LLMAgentic'...
remote: Enumerating objects: 235, done.
remote: Counting objects: 100% (235/235), done.
remote: Compressing objects: 100% (173/173), done.
remote: Total 235 (delta 91), reused 181 (delta 50), pack-reused 0 (from 0)
Receiving objects: 100% (235/235), 25.72 MiB | 11.23 MiB/s, done.
Resolving deltas: 100% (91/91), done.


In [ ]:
import json
import os

# Assuming knowledgeBase.json is in the specified path relative to the current working directory
file_path = "OfficeBuddyPrime_LLMAgentic/data/knowledgeBase.json"

def read_json_file(file_path):

  try:
    with open(file_path, 'r') as f:
      sample_records = json.load(f)

  except FileNotFoundError:
    print(f"Error: File not found at '{file_path}'. Please check the file path.")
  except json.JSONDecodeError:
    print(f"Error: Invalid JSON format in '{file_path}'. Please check the file content.")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")

  return sample_records

sample_records = read_json_file(file_path)['records']
print(sample_records)


[{'id': 1, 'question': 'ใครมีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?', 'answer': 'ใครมีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา? ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะ อย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชน ตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญ หรือคณะบุคคลที่มิใช่นิติบุคคล ปรับปรุงล่าสุด: 21-11-2020'}, {'id': 2, 'question': 'เมื่อมีเงินได้เกิดขึ้นแล้ว ผู้มีหน้าที่เสียภาษีจะต้องทำอะไรบ้าง?', 'answer': 'เมื่อมีเงินได้เกิดขึ้นแล้ว ผู้มีหน้าที่เสียภาษีจะต้องทำอะไรบ้าง? ขอมีเลข และบัตรประจำตัวผู้เสียภาษี ภายใน 60 วัน นับแต่วันที่มีเงินได้เกิดขึ้น กรณีเป็นผู้มีเงินได้ ที่ไม่มีเลขประจำตัวประชาชน ได้แก่ เป็นคนต่างด้าว หรือกองมรดกที่ยังไม่ได้แบ่ง เว้นแต่ ผู้มีเงินได้ ที่มีเลขประจำตัวประชาชน สามารถใช้ เลขประจำตัวประชาชน แทนเลขประจำตัว ผู้เสียภาษีอากรได้ โดยไม่ต้องขอมีเลขประจำตัวผู้เสียภาษีอ

In [ ]:
from langchain_core.documents import Document

documents = [ Document(page_content=document["answer"], metadata={"id":document["id"]}) for document in sample_records]
id_question = [ (document["id"], document['question']) for document in sample_records]

In [ ]:
import os
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import ParentDocumentRetriever

embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3")


<ipython-input-10-6aa89f6e69f8>:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-m3")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
vectorstores = FAISS.from_documents(documents, embeddings)
retriever = vectorstores.as_retriever()
#

In [ ]:
retriever_results = []
relevant_documents = []
for id, question in id_question :
  results = retriever.invoke(question)
  retrieved_id  = [doc.metadata['id'] for doc in results]
  retriever_results.append(retrieved_id)
  relevant_documents.append([id])


In [ ]:
# prompt: Write me score MRR from previous retriever and document

import numpy as np

def calculate_mrr(retriever_results, relevant_documents):
    """
    Calculates the Mean Reciprocal Rank (MRR) for a set of queries.

    Args:
        retriever_results: A list of lists, where each inner list contains the ranked IDs
                           of documents retrieved for a query.
        relevant_documents: A list of lists, where each inner list contains the IDs of the
                            relevant documents for a query.

    Returns:
        The MRR score.
    """

    mrr_sum = 0
    for i in range(len(retriever_results)):
        try:
            rank = retriever_results[i].index(relevant_documents[i][0]) + 1 # Find the rank of first relevant doc
            mrr_sum += 1.0 / rank
        except ValueError:  # Handle cases where no relevant document is found
            pass  # or mrr_sum += 0, depending on how you want to handle this

    return mrr_sum / len(retriever_results) if retriever_results else 0

mrr = calculate_mrr(retriever_results, relevant_documents)
print(f"MRR: {mrr}")


MRR: 0.9166666666666666


In [ ]:

tool = create_retriever_tool(
    retriever,
    "search_tax_information",
    "Searches and returns information about text in thai language.",
)
tools = [tool]

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

In [ ]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="gpt-4.1-mini-2025-04-14",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    temperature=0,
    max_tokens=2500,
    timeout=None,
    max_retries=2,
)


In [ ]:
from functools import partial

agent = create_react_agent(llm, tools=tools)
def ask_agent(question, agent_instance):
    return agent_instance.invoke({"messages": [{"role": "user", "content": question}]})

# Create a partial function with the agent already provided
agent_call = partial(ask_agent, agent_instance=agent)

In [ ]:

def get_metric(answer) :

  def get_usage(key: str, message) -> int :
    return message.response_metadata.get("usage", {}).get(key, 0)

  def token_to_usd(input_token : int, output_token : int , input_per_million = 3, output_per_million = 15) :
    input_price = input_per_million / 1000000
    output_price = output_per_million / 1000000
    return (input_token * input_price + output_token * output_price)

  input_token = 0
  output_token = 0
  for ret in answer['messages'] :
    input_token += get_usage("input_tokens", ret)
    output_token += get_usage("output_tokens", ret)

  price = token_to_usd(input_token=input_token, output_token=output_token)

  return { "input_tokens": input_token, "output_tokens": output_token, "cost_usd":price}

get_metric(answer)

TypeError: string indices must be integers, not 'str'

In [ ]:
answer["messages"][-1].pretty_print()

TypeError: string indices must be integers, not 'str'

In [ ]:
test_path = "OfficeBuddyPrime_LLMAgentic/testset/tax-test-set.json"

In [ ]:
testset = read_json_file(test_path)['test_cases']

In [ ]:
print(testset)

[{'id': 1, 'question': 'ใครบ้างที่มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา?', 'golden_answer': 'ผู้มีหน้าที่เสียภาษีเงินได้บุคคลธรรมดา ได้แก่ ผู้ที่มีเงินได้เกิดขึ้นระหว่างปีที่ผ่านมาโดยมีสถานะอย่างหนึ่งอย่างใด ดังนี้ 1) บุคคลธรรมดา 2) ห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล 3) ผู้ถึงแก่ความตายระหว่างปีภาษี 4) กองมรดกที่ยังไม่ได้แบ่ง 5) วิสาหกิจชุมชนตามกฎหมายว่าด้วยการส่งเสริมวิสาหกิจชุมชน เฉพาะที่เป็นห้างหุ้นส่วนสามัญหรือคณะบุคคลที่มิใช่นิติบุคคล'}, {'id': 2, 'question': 'เงินได้ประเภทใดบ้างที่ต้องเสียภาษีเงินได้บุคคลธรรมดา?', 'golden_answer': 'เงินได้พึงประเมินที่ต้องเสียภาษีเงินได้บุคคลธรรมดา แบ่งเป็น 8 ประเภท ดังนี้: 1. เงินได้ประเภทที่ 1: เงินเดือน ค่าจ้าง เบี้ยเลี้ยง โบนัส และเงินอื่นๆ จากการจ้างแรงงาน 2. เงินได้ประเภทที่ 2: เงินได้จากหน้าที่หรือตำแหน่งงาน เช่น ค่าธรรมเนียม ค่านายหน้า 3. เงินได้ประเภทที่ 3: ค่าแห่งกู๊ดวิลล์ ค่าลิขสิทธิ์ เงินปี 4. เงินได้ประเภทที่ 4: ดอกเบี้ย เงินปันผล เงินส่วนแบ่งกำไร ผลประโยชน์จากการโอนหุ้น 5. เงินได้ประเภทที่ 5: เงินได้จากการให้เช่าทรัพย์สิน 

In [ ]:
import time
results = []
for items in testset[:] :
  id, question, gold_answer = items.values()
  start = time.time()
  answer = agent_call(question)
  stop = time.time()
  metrics = get_metric(answer)
  metrics['time_seconds'] = stop - start
  results.append({
    "id": id,
    "question": question,
    "inference_result": answer["messages"][-1].content,
    "gold_answer": gold_answer,
    "metrics": metrics
  })


In [ ]:
json.dumps({
  "description": "Thai Personal Income Tax RAG Test Set for evaluating retrieval and QA performance results from BGE-M3 with Claude 3.7 sonnet",
  "version": "1.0",
  "date_created": "2025-05-08",
  "test_cases": results
})

'{"description": "Thai Personal Income Tax RAG Test Set for evaluating retrieval and QA performance results from BGE-M3 with Claude 3.7 sonnet", "version": "1.0", "date_created": "2025-05-08", "test_cases": [{"id": 1, "question": "\\u0e43\\u0e04\\u0e23\\u0e1a\\u0e49\\u0e32\\u0e07\\u0e17\\u0e35\\u0e48\\u0e21\\u0e35\\u0e2b\\u0e19\\u0e49\\u0e32\\u0e17\\u0e35\\u0e48\\u0e40\\u0e2a\\u0e35\\u0e22\\u0e20\\u0e32\\u0e29\\u0e35\\u0e40\\u0e07\\u0e34\\u0e19\\u0e44\\u0e14\\u0e49\\u0e1a\\u0e38\\u0e04\\u0e04\\u0e25\\u0e18\\u0e23\\u0e23\\u0e21\\u0e14\\u0e32?", "inference_result": "\\u0e08\\u0e32\\u0e01\\u0e02\\u0e49\\u0e2d\\u0e21\\u0e39\\u0e25\\u0e17\\u0e35\\u0e48\\u0e04\\u0e49\\u0e19\\u0e2b\\u0e32\\u0e44\\u0e14\\u0e49 \\u0e1c\\u0e39\\u0e49\\u0e21\\u0e35\\u0e2b\\u0e19\\u0e49\\u0e32\\u0e17\\u0e35\\u0e48\\u0e40\\u0e2a\\u0e35\\u0e22\\u0e20\\u0e32\\u0e29\\u0e35\\u0e40\\u0e07\\u0e34\\u0e19\\u0e44\\u0e14\\u0e49\\u0e1a\\u0e38\\u0e04\\u0e04\\u0e25\\u0e18\\u0e23\\u0e23\\u0e21\\u0e14\\u0e32 \\u0e44\\u0e14\\u0e49

In [ ]:


import json

output_data = {
  "description": "Thai Personal Income Tax RAG Test Set for evaluating retrieval and QA performance results from BGE-M3 with Claude 3.7 sonnet",
  "version": "1.0",
  "date_created": "2025-05-08",
  "test_cases": results
}

# Specify the desired file path
output_file_path = "evaluation_results.json"

try:
  with open(output_file_path, 'w') as f:
    json.dump(output_data, f, indent=4)  # Use indent for pretty printing
  print(f"Data successfully saved to {output_file_path}")

except Exception as e:
  print(f"An error occurred while saving the file: {e}")


Data successfully saved to evaluation_results.json
